<a href="https://colab.research.google.com/github/2002sairuthvik/Fine_Tuning/blob/main/finetune_gpro_gemma_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
if "COLAB_" not in "".join(os.environ.keys()):
  !pip install unsloth vllm
else:
  !pip install --no-deps unsloth vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.8/299.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 MB 2.7 MB/s eta 0:00:00


In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm==0.8.5.post1
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B-Base",
    max_seq_length = max_seq_length,
    load_in_4bit = False, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.7, # Reduce if out of memory
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 08-03 13:48:38 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 08-03 13:48:38 [__init__.py:239] Automatically detected platform cuda.
Unsloth: Patching vLLM v1 graph capture
Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.8.1: Fast Qwen3 patching. Transformers: 4.54.0. vLLM: 0.8.5.post1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/Qwen3-4B-Base with actual GPU utilization = 69.2%
Unsloth: Your GPU has CUDA compute capability 8.0

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2, # *2 speeds up training
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    random_state = 3407,
)

Unsloth 2025.8.1 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [5]:
reasoning_start = "<start_working_out>" # Acts as <think>
reasoning_end   = "<end_working_out>"   # Acts as </think>
solution_start  = "<SOLUTION>"
solution_end    = f"</SOLUTION>{tokenizer.eos_token}"

system_prompt = \
f"""You are given a problem.
Think about the problem and provide your working out.
Place it between {reasoning_start} and {reasoning_end}.
Then, provide your solution between {solution_start}{solution_end}"""
system_prompt

'You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION><|endoftext|>'

In [6]:
from datasets import load_dataset

dataset = load_dataset("open-r1/DAPO-Math-17k-Processed","en",split="train")
dataset

README.md: 0.00B [00:00, ?B/s]

en/train-00000-of-00001.parquet:   0%|          | 0.00/5.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14116 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'solution', 'data_source', 'source_prompt', 'ability', 'reward_model', 'extra_info'],
    num_rows: 14116
})

In [7]:
dataset[0]['prompt']

'In triangle $ABC$, $\\sin \\angle A = \\frac{4}{5}$ and $\\angle A < 90^\\circ$. Let $D$ be a point outside triangle $ABC$ such that $\\angle BAD = \\angle DAC$ and $\\angle BDC = 90^\\circ$. Suppose that $AD = 1$ and that $\\frac{BD}{CD} = \\frac{3}{2}$. If $AB + AC$ can be expressed in the form $\\frac{a\\sqrt{b}}{c}$ where $a, b, c$ are pairwise relatively prime integers, find $a + b + c$.'

In [8]:
dataset[0]['solution']

'34'

In [9]:
dataset[0]['reward_model']

{'ground_truth': '34', 'style': 'rule-lighteval/MATH_v2'}

In [10]:
def extract_hash_answer(text):
  return text

extract_hash_answer(dataset[0]['solution'])

'34'

In [11]:
dataset = dataset.map(lambda x :{
    "prompt" : [
        {"role":'system',"content":system_prompt,},
        {"role":'user',"content":x['prompt'],}
    ],
    "answer" : extract_hash_answer(x['solution']),
})
dataset[0]

Map:   0%|          | 0/14116 [00:00<?, ? examples/s]

{'prompt': [{'content': 'You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION><|endoftext|>',
   'role': 'system'},
  {'content': 'In triangle $ABC$, $\\sin \\angle A = \\frac{4}{5}$ and $\\angle A < 90^\\circ$. Let $D$ be a point outside triangle $ABC$ such that $\\angle BAD = \\angle DAC$ and $\\angle BDC = 90^\\circ$. Suppose that $AD = 1$ and that $\\frac{BD}{CD} = \\frac{3}{2}$. If $AB + AC$ can be expressed in the form $\\frac{a\\sqrt{b}}{c}$ where $a, b, c$ are pairwise relatively prime integers, find $a + b + c$.',
   'role': 'user'}],
 'solution': '34',
 'data_source': 'math_dapo',
 'source_prompt': [{'content': 'Solve the following math problem step by step. The last line of your response should be of the form Answer: $Answer (without quotes) where $Answer is the answer to the problem.\n\nIn triangle $ABC$, $\\sin \\angle A = \\frac

In [12]:
import re

solution_end_regex = r"</SOLUTION>[\s]{0,}" +\
  "(?:" + re.escape(tokenizer.eos_token) + ")?"

match_format = re.compile(
    rf"{reasoning_end}.*?"\
    rf"{solution_start}(.+?){solution_end_regex}"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)
match_format

re.compile(r'<end_working_out>.*?<SOLUTION>(.+?)</SOLUTION>[\s]{0,}(?:<\|endoftext\|>)?[\s]{0,}$',
re.MULTILINE|re.DOTALL|re.UNICODE)

In [13]:
match_format.findall(
    "Let me think!<end_working_out>"\
    f"<SOLUTION>\n2\n</SOLUTION>",
)

['\n2\n']

In [14]:
match_format.findall(
    "<start_working_out>Let me think!<end_working_out>"\
    f"<SOLUTION> 2 </SOLUTION>\n\n",
)

[' 2 ']

In [27]:
def match_format_exactly(completions,**kwargs):
  scores=[]
  for completion in completions:
    score=0
    response = completion[0]['content']
    if match_format.search(response) is not None: score +=3.0
    scores.append(score)
  return scores

In [16]:
def match_format_approximately(completions,**kwargs):
  scores=[]
  for completion in completions:
    score=0
    response = completion[0]['content']
    # count how many keywords are seen - we penalize if too many!
    #if we see 1 , then plus some points!

    # no need to reward <start_working_out> sisnce we always prepend it!
    # score+=0.5 if response.count(reasoning_start)==1 else -1.0
    score += 0.5 if response.count(reasoning_end)==1 else -1.0
    score += 0.5 if response.count(solution_start)==1 else -1.0
    score += 0.5 if response.count(solution_end)==1 else -1.0
    scores.append(score)
  return scores

In [17]:
def check_answer(prompts,completions,answer,**kwargs):
  question = prompts[0][-1]['content']
  responses = [completion[0]['content'] for completion in completions]

  extracted_responses = [
      guess.group(1)
      if (guess := match_format.search(r)) is not None else None\
      for r in responses
  ]

  scores = []
  for guess,true_answer in zip(extracted_responses,answer):
    score=0
    if guess is None:
      scores.append(-2.0)
      continue
    # correct gets 5 points
    if guess == true_answer:
      score +=5.0
    # match if spaces are seen, but less reward
    elif guess.strip() == true_answer.strip():
      score+=3.5
    else:
      # we also reward it if answer is close via ratios i.e if answer is with in some range, reward it!
      try:
        ratio = float(guess) / float(true_answer)
        if ratio>=0.9 and ratio<=1.1: score +=2.0
        elif ratio>=0.8 and ratio<=1.2: score +=1.5
        else: score -=2.5 # penalise wrong answers
      except:
        score -=4.5 # peanlize
    scores.append(score)
  return scores

In [18]:
match_numbers = re.compile(
    solution_start + r".*?[\s]{0,}([-]?[\d\.\,]{1,})",
    flags = re.MULTILINE | re.DOTALL


)
print(match_numbers.findall("<SOLUTION>   0.34  </SOLUTION>"))
print(match_numbers.findall("<SOLUTION>   123,456  </SOLUTION>"))
print(match_numbers.findall("<SOLUTION>   -0.234  </SOLUTION>"))
print(match_numbers.findall("<SOLUTION>17</SOLUTION>"))

['0.34']
['123,456']
['-0.234']
['17']


In [30]:
global PRINTED_TIMES
PRINTED_TIMES = 0
global PRINT_EVERY_STEPS
PRINT_EVERY_STEPS = 5

def check_numbers(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_numbers.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    # Print only every few steps
    global PRINTED_TIMES
    global PRINT_EVERY_STEPS
    if PRINTED_TIMES % PRINT_EVERY_STEPS == 0:
        print(
            '*'*20 + f"Question:\n{question}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}"
        )
    PRINTED_TIMES += 1

    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None:
            scores.append(-2.5)
            continue
        # Convert to numbers
        try:
            true_answer = float(true_answer.strip())
            # Remove commas like in 123,456
            guess       = float(guess.strip().replace(",", ""))
            scores.append(3.5 if guess == true_answer else -1.5)
        except:
            scores.append(0)
            continue
    return scores

In [20]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-4B-Base")
print(tokenizer.chat_template)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

{%- if tools %}
    {{- '<|im_start|>system\n' }}
    {%- if messages[0].role == 'system' %}
        {{- messages[0].content + '\n\n' }}
    {%- endif %}
    {{- "# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>" }}
    {%- for tool in tools %}
        {{- "\n" }}
        {{- tool | tojson }}
    {%- endfor %}
    {{- "\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{\"name\": <function-name>, \"arguments\": <args-json-object>}\n</tool_call><|im_end|>\n" }}
{%- else %}
    {%- if messages[0].role == 'system' %}
        {{- '<|im_start|>system\n' + messages[0].content + '<|im_end|>\n' }}
    {%- endif %}
{%- endif %}
{%- set ns = namespace(multi_step_tool=true, last_query_index=messages|length - 1) %}
{%- for message in messages[::-1] %}
    {%- set index = (messages|length - 

In [21]:
tokenized = dataset.map(
    lambda x:{"tokens" : tokenizer.apply_chat_template(x['prompt'],add_generation_prompt=True, tokenize=True)},
    batched=True
)
print(tokenizer.decode(tokenized[0]['tokens']))
tokenized  = tokenized.map(lambda x: {"L":len(x["tokens"])})

import numpy as np
maximum_length = int(np.quantile(tokenized['L'],0.9))
print("Max-Length = ", maximum_length)

# filter only samples smaller than 90% max length
dataset = dataset.select (np.where(np.array(tokenized['L'])<=maximum_length)[0])
del tokenized

Map:   0%|          | 0/14116 [00:00<?, ? examples/s]

<|im_start|>system
You are given a problem.
Think about the problem and provide your working out.
Place it between <start_working_out> and <end_working_out>.
Then, provide your solution between <SOLUTION></SOLUTION><|endoftext|><|im_end|>
<|im_start|>user
In triangle $ABC$, $\sin \angle A = \frac{4}{5}$ and $\angle A < 90^\circ$. Let $D$ be a point outside triangle $ABC$ such that $\angle BAD = \angle DAC$ and $\angle BDC = 90^\circ$. Suppose that $AD = 1$ and that $\frac{BD}{CD} = \frac{3}{2}$. If $AB + AC$ can be expressed in the form $\frac{a\sqrt{b}}{c}$ where $a, b, c$ are pairwise relatively prime integers, find $a + b + c$.<|im_end|>
<|im_start|>assistant



Map:   0%|          | 0/14116 [00:00<?, ? examples/s]

Max-Length =  210


In [22]:
max_prompt_length = maximum_length + 1 # +1 just in case
max_completion_length = max_seq_length - max_prompt_length

In [23]:
from vllm import SamplingParams
vllm_sampling_params = SamplingParams(
    min_p = 0.1,
    top_p = 1.0,
    top_k = -1,
    seed = 3407,
    stop = [tokenizer.eos_token],
    include_stop_str_in_output = True,
)

In [24]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    vllm_sampling_params = vllm_sampling_params,
    temperature = 1.0,
    learning_rate = 5e-6,
    weight_decay = 0.01,
    warmup_ratio = 0.1,
    lr_scheduler_type = "linear",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_completion_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 100,
    save_steps = 100,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",

    # For optional training + evaluation
    # fp16_full_eval = True,
    # per_device_eval_batch_size = 4,
    # eval_accumulation_steps = 1,
    # eval_strategy = "steps",
    # eval_steps = 1,
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


In [31]:
# For optional training + evaluation
# new_dataset = dataset.train_test_split(test_size = 0.01)

trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        match_format_exactly,
        match_format_approximately,
        check_answer,
        check_numbers,
    ],
    args = training_args,
    train_dataset = dataset,

    # For optional training + evaluation
    # train_dataset = new_dataset["train"],
    # eval_dataset = new_dataset["test"],
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 12,728 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 66,060,288 of 4,088,528,384 (1.62% trained)


********************Question:
In the diagram, each of the three identical circles touch the other two.  The circumference of each circle is 36.  What is the perimeter of the shaded region? [asy]

defaultpen(1);

path p = (1, 0){down}..{-dir(30)}dir(-60){dir(30)}..{dir(-30)}((2, 0) + dir(-120)){-dir(-30)}..{up}(1, 0)--cycle;
fill(p, gray(0.75));

draw(unitcircle);
draw(shift(2 * dir(-60)) * unitcircle);
draw(shift(2) * unitcircle);
[/asy] 
Answer:
18 
Response:
In the diagram, each of the three identical circles touch the other two.  The circumference of each circle is 36.

We are to find the perimeter of the shaded region.

Since all three circles have the same radius, let's denote the radius of each circle by $r$.

The circumference of a circle is given by the formula $C = 2\pi r$.

Given that the circumference of each circle is 36, we have $2\pi r = 36$.

Solving for $r$, we find that $r = \frac{36}{2\pi} = \frac{18}{\pi}$.

The shaded region consists of three arcs: one for each circ

Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,entropy,rewards / match_format_exactly / mean,rewards / match_format_exactly / std,rewards / match_format_approximately / mean,rewards / match_format_approximately / std,rewards / check_answer / mean,rewards / check_answer / std,rewards / check_numbers / mean,rewards / check_numbers / std
1,0.000000,-6.250000,1.443376,324.500000,33.000000,536.000000,0.000000,324.500000,33.000000,536.000000,0.000000,0,0.000000,0.000000,-2.250000,0.866025,-2.000000,0.000000,-2.000000,0.577350
2,-0.000000,-5.000000,3.000000,1600.000000,889.000000,1837.000000,0.750000,889.000000,889.000000,889.000000,0.000000,No Log,0.750000,1.500000,-1.875000,1.436141,-2.625000,1.250000,-1.250000,1.443376
3,0.000000,-6.125000,1.887459,1254.250000,593.000000,1837.000000,0.250000,1060.000000,593.000000,1307.000000,0.001100,No Log,0.000000,0.000000,-2.250000,0.866025,-2.000000,0.000000,-1.875000,1.250000
4,0.000000,-5.375000,0.750000,1058.500000,297.000000,1837.000000,0.250000,799.000000,297.000000,1396.000000,0.000909,No Log,0.000000,0.000000,-1.875000,0.750000,-2.000000,0.000000,-1.500000,0.000000
5,0.000000,-4.000000,4.062019,634.000000,4.000000,1837.000000,0.250000,233.000000,4.000000,576.000000,0.001106,No Log,0.750000,1.500000,-1.875000,1.436141,-2.125000,0.250000,-0.750000,2.872281
6,0.000000,-5.875000,1.181454,598.500000,205.000000,977.000000,0.000000,598.500000,205.000000,977.000000,0.000717,No Log,0.000000,0.000000,-1.875000,0.750000,-2.000000,0.000000,-2.000000,0.577350
7,0.000000,-5.625000,3.119161,989.750000,342.000000,1837.000000,0.250000,707.333374,342.000000,911.000000,0.000655,No Log,0.750000,1.500000,-2.250000,1.500000,-2.125000,0.250000,-2.000000,0.577350
8,0.000000,-7.500000,0.000000,1142.250000,255.000000,1837.000000,0.250000,910.666687,255.000000,1726.000000,0.000857,No Log,0.000000,0.000000,-3.000000,0.000000,-2.000000,0.000000,-2.500000,0.000000
9,0.000000,-5.750000,2.179450,771.500000,18.000000,1174.000000,0.000000,771.500000,18.000000,1174.000000,0.000894,No Log,0.750000,1.500000,-1.875000,1.436141,-2.625000,1.250000,-2.000000,0.577350
10,0.000000,-5.875000,1.973786,1273.500000,894.000000,1837.000000,0.250000,1085.666748,894.000000,1458.000000,0.000888,No Log,0.000000,0.000000,-1.875000,1.436141,-2.000000,0.000000,-2.000000,0.577350


********************Question:
In a class of 20 students, all but 4 of the students put their names on a typed assignment. If the teacher randomly guesses, what is the probability that she correctly guesses which paper belongs to each of the four remaining students? Express your answer as a common fraction.The answer is in the form rac{m}{n}, where gcd(m, n) = 1. Please provide the value of m + n. 
Answer:
25 
Response:
ูก่อนตอบคำถามค่า
orea
<start_working_out>
We are trying to find the probability of the teacher correctly guessing which paper belongs to each of the four remaining students. Since there are 4 students, we can assume that the teacher has 4 random choices for each student. Therefore, the total number of possible outcomes is 4! = 24. 
 
To find the number of successful outcomes where the teacher correctly guesses each student's paper, we need to find how many ways we can arrange the 4 papers among the 4 students. This can be done in 4! = 24 ways as well. 
 
Therefore, the 

TrainOutput(global_step=100, training_loss=7.94107251849141e-07, metrics={'train_runtime': 2533.7108, 'train_samples_per_second': 0.158, 'train_steps_per_second': 0.039, 'total_flos': 0.0, 'train_loss': 7.94107251849141e-07})

In [32]:
text = "What is the sqrt of 101?"

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

' To find the square root of 101, we can use the following approximate value: √101 ≈ 10.04987562112.'

In [34]:
import os

hf_token = ""  # Add your actual HF token or ensure it's fetched securely
os.environ["HF_TOKEN"] = hf_token

# Save the merged 16-bit model and tokenizer locally
if True:
    model.save_pretrained_merged("model", tokenizer, save_method="merged_16bit")

# Push to your Hugging Face repo (replace "gemma-3-16bit" with the desired model name)
if True:
    model.push_to_hub_merged(
        "ruthvi29/finetune_gpro_gemma-3-16bit",  # Place your target Hugging Face repo path here
        tokenizer,
        save_method="merged_16bit",
        token=hf_token
    )


Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Successfully copied all 2 files from cache to model.


Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [00:49<00:00, 24.99s/it]


ValueError: Could not determine original model ID from None. If using a local model, ensure the path exists and contains safetensors files.